In [1]:
import sys
import os
import warnings

# DO NOT remove this line below, otherwise the app won't be found
sys.path.append('..')
warnings.filterwarnings('ignore')

In [2]:
import param

import io
import panel as pn
import numpy as np
import pandas as pd

import sim2bids
from sim2bids.sim2bids import MainArea

pn.extension('tabulator', 'ace', 'jsoneditor', 'ipywidgets', sizing_mode='stretch_width', notifications=True)

NotificationArea(sizing_mode='stretch_width')

In [3]:
path = '../output'
sys.path.append(path)

sim2bids.app.app.SoftwareVersion = 2.6
sim2bids.app.app.SoftwareRepository = 'https://github.com/the-virtual-brain/tvb-root/releases/tag/2.6'
sim2bids.app.app.SoftwareName = 'TVB'

app = MainArea()
pn.serve(app.view())

INFO:bokeh.server.server:Starting Bokeh server version 2.4.3 (running on Tornado 6.1)
INFO:bokeh.server.tornado:User authentication hooks NOT provided (default user enabled)


Launching server at http://localhost:62565


In [7]:
import sim2bids.app.utils as utils

utils.get_content('output', os.listdir('output'))

['output\\CHANGES.txt',
 'output\\code\\desc-default_code.json',
 'output\\code\\desc-default_code.py',
 'output\\coord\\desc-default_labels.json',
 'output\\coord\\desc-default_labels.tsv',
 'output\\coord\\desc-default_nodes.json',
 'output\\coord\\desc-default_nodes.tsv',
 'output\\coord\\desc-default_times.json',
 'output\\coord\\desc-default_times.tsv',
 'output\\eq\\desc-default_eq.json',
 'output\\eq\\desc-default_eq.xml',
 'output\\param\\desc-default_param.json',
 'output\\param\\desc-default_param.xml',
 'output\\README.txt',
 'output\\sub-01\\net\\sub-01_desc-default_distances.json',
 'output\\sub-01\\net\\sub-01_desc-default_distances.tsv',
 'output\\sub-01\\net\\sub-01_desc-default_weights.json',
 'output\\sub-01\\net\\sub-01_desc-default_weights.tsv',
 'output\\sub-01\\spatial\\sub-01_desc-default_fc.json',
 'output\\sub-01\\spatial\\sub-01_desc-default_fc.tsv',
 'output\\sub-01\\ts\\sub-01_desc-default_bold.json',
 'output\\sub-01\\ts\\sub-01_desc-default_bold.tsv',
 'ou

In [30]:
import sim2bids.convert.convert as convert
import json

OUTPUT = 'output'

def check_json():
    for file in utils.get_content(OUTPUT, os.listdir(OUTPUT)):
        if file.endswith('json'):
            f = json.load(open(file))
            if 'CoordRows' in f.keys() and f['CoordRows'] == '':
                f['CoordRows'] = convert.COORDS
                f['CoordColumns'] = convert.COORDS
            
            with open(file, 'w') as f2:
                json.dump(f, f2)
            

In [31]:
check_json()

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [5]:
accepted('bold.txt')

name: bold.txt


True

In [11]:
sim2bids.app.app.ACCEPTED

['weight',
 'distance',
 'tract_length',
 'delay',
 'speed',
 'nodes',
 'labels',
 'centres',
 'area',
 'hemisphere',
 'cortical',
 'orientation',
 'average_orientation',
 'normal',
 'times',
 'vertices',
 'faces',
 'vnormal',
 'fnormal',
 'sensor',
 'map',
 'volume',
 'cartesian2d',
 'cartesian3d',
 'polar2d',
 'polar3d',
 'vars',
 'stimuli',
 'noise',
 'spike',
 'raster',
 'ts',
 'event',
 'emp',
 'boldfc']